In [1]:
from truss_casadi import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

joints = [
    Joint(JointType.PIN),  # 0
    Joint(JointType.GUSSET), # 1
    Joint(JointType.GUSSET), # 2
    Joint(JointType.GUSSET), # 3
    Joint(JointType.ROLLER), # 4 
    Joint(JointType.GUSSET), # 5
    Joint(JointType.GUSSET), # 6
    Joint(JointType.GUSSET), # 7
    Joint(JointType.GUSSET), # 8
    Joint(JointType.GUSSET)] # 9


links = [Link(0, 1), Link(1,2), Link(2,3), Link(3,4), Link(0,5), Link(1,5), Link(1,6), Link(2,6), Link(2,7), Link(2,8), Link(3,8), Link(3,9), Link(4,9), Link(5,6), Link(6,7), Link(7,8), Link(8,9)]

FORCE_PER_M = 2.5
bridge_joints = [0, 1, 2, 3, 4]
train_force = Force(fx=0, fy=FORCE_PER_M, joints=bridge_joints)

forces = [train_force]

truss = Truss(joints, links, forces)


In [2]:
import math

half_len_unknowns = len(joints)

inputs = [-1]*half_len_unknowns*2
inputs[bridge_joints[0]] = 0
inputs[bridge_joints[-1]] = 14
for i in bridge_joints:
    inputs[i + half_len_unknowns] = 0

cost_ = Function('cost_', [truss.xs], [truss.cost])

def constraints():
    inequalities = []
    inequality_funs = []
    inequality_gradients = []
    for link in truss.links:
        leng_fun = Function('leng_fun', [truss.xs], [link.length])
        force_fun = Function('force_fun', [truss.xs], [link.force])   
        # Constraint: length of each link cannot be less than 1
        # inequalities.append(-leng_fun(truss.xs)+1)
        inequalities.append(-link.length + 1)
        inequality_funs.append(Function('length_greater_than_one', [truss.xs], [-link.length + 1]))
        inequality_gradients.append(Function('del_length_greater_than_one', [truss.xs], [casadi.gradient(-link.length + 1, truss.xs)]))
        # Constraint: force of each link must be within -27 and 18
        inequalities.append(link.force - 6*3)
        inequality_funs.append(Function('force_less_than_18', [truss.xs], [link.force - 6*3]))
        # inequalities.append(force_fun(truss.xs) - 6*3)
        inequalities.append(-link.force - 9*3)
        inequality_funs.append(Function('force_greater_than_27', [truss.xs], [-link.force - 9*3]))
        # inequalities.append(-force_fun(truss.xs) - 9*3)


    for i in range(len(bridge_joints)-1):
        i0 = bridge_joints[i]
        i1 = bridge_joints[i+1]

        # Constraint: Each bridge joint must not be more than 3.5m apart
        # inequalities.append(truss.xs[i1] - truss.xs[i0] - 3.5)
        inequalities.append(truss.xs[i1] - truss.xs[i0] - 3.5)
        inequality_funs.append(Function('bridge_spacing', [truss.xs], [truss.xs[i1] - truss.xs[i0] - 3.5]))

        # Constraint: Bridge joints must be in order of ascending X value
        # inequalities.append(truss.xs[i0] - truss.xs[i1])
        inequalities.append(truss.xs[i0] - truss.xs[i1])
        inequality_funs.append(Function('bridge_ascending', [truss.xs], [truss.xs[i0] - truss.xs[i1]]))

    return inequalities, inequality_funs

inequalities, inequality_funs = constraints()
del_ineq_fn = Function('del_ineq_fn', [truss.xs], [casadi.gradient(casadi.norm_2(casadi.fmax(casadi.horzcat(*inequalities), 0)), truss.xs)])


In [3]:
# def populate_unknowns(x):
#     if(len(x.size()) == 2):
#         final = []
#         for j in range(x.size()[0]):
#             x_iter = iter(x[j])
#             xs = inputs.copy()
#             for i in range(len(inputs)):
#                 xs[i] = inputs[i] if inputs[i] != -1 else next(x_iter)
#             final.append(xs)
#         return torch.tensor(final, requires_grad=True).to(DEVICE)
#     else:
#         x_iter = iter(x)
#         xs = inputs.copy()
#         for i in range(len(inputs)):
#             xs[i] = inputs[i] if inputs[i] != -1 else next(x_iter)
#         return torch.tensor(xs, requires_grad=True).to(DEVICE)

# def extract_unknowns(x):
#     if(len(x.size()) == 2):
#         all_xs = []
#         for j in range(len(x)):
#             xs = []
#             for i in range(len(inputs)):
#                 if inputs[i] == -1:
#                     print(x[j])
#                     n = float(x[j][i])
#                     if math.isnan(n):
#                         n = 0.0
#                     xs.append(n)   
#             all_xs.append(xs)
#         return torch.tensor(all_xs, requires_grad=True).to(DEVICE) 
#     else:
#         xs = []
#         for i in range(len(inputs)):
#             if inputs[i] == -1:
#                 n = float(x[i])
#                 if math.isnan(n):
#                     n = 0.0
#                 xs.append(n)
#         return torch.tensor(xs, requires_grad=True).to(DEVICE)
    

# def cost_fn(y):
#     return float(cost_(populate_unknowns(y).cpu().detach().numpy().flatten()))

# # inequality function g_x(y) <= 0
# def g_x(y):
#     x = populate_unknowns(y)
#     G = []
#     for eqn in inequality_funs:
#         # print(eqn, "=", eqn(x))
#         G.append([float(eqn(x.cpu().detach().numpy().flatten()))])
#     return torch.tensor(G, requires_grad=True).to(DEVICE)

# del_ineq_fn = Function('del_ineq_fn', [truss.xs], [casadi.gradient(casadi.norm_2(casadi.fmax(casadi.vertcat(*inequalities), 0)), truss.xs)])
# def del_ineq(y):
#     if(len(y.size()) == 2):
#         xs = []
#         for i in range(len(y)):
#             xs.append(extract_unknowns(torch.tensor(np.array(del_ineq_fn(populate_unknowns(y[i]).cpu().detach().numpy().flatten())).flatten(),requires_grad=True).to(DEVICE)).cpu().detach().numpy().flatten())
#         return torch.tensor(xs, requires_grad=True).to(DEVICE)
#     else:
#         return extract_unknowns(torch.tensor(np.array(del_ineq_fn(populate_unknowns(y).cpu().detach().numpy().flatten())).flatten(), requires_grad=True).to(DEVICE))


# ground_truth = torch.tensor([3.5, 7, 10.5, -6.6+7, -4.1+7, 0, 4.9+7, 6.4+7, -1.9, -4, -4.9, -4, -1.9]).to(DEVICE)

# print(populate_unknowns(ground_truth))

# n_unknowns = half_len_unknowns*2-len(bridge_joints)-2

# # print(del_g_x(ground_truth))
# # print(torch.linalg.vector_norm(torch.clamp(del_g_x(ground_truth), min=0), dim=1))

# print(g_x(ground_truth))
# print(del_ineq(ground_truth))
# # cost_fn(ground_truth)
# # g_x(ground_truth)

In [4]:
ground_truth_with_knowns = torch.tensor([[0, 3.5, 7, 10.5, 14, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, 0, 0, 0, 0, 0, -1.9, -4, -4.9, -4, -1.9]]).to(DEVICE)
ground_truth = torch.tensor([[3.5, 7, 10.5, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, -1.9, -4, -4.9, -4, -1.9]]).to(DEVICE)
known_indices = [1, 2, 3, 5, 6, 7, 8, 9, 15, 16, 17, 18, 19]
replace_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
n_unknowns = 13

def remove_unknowns(X):
    return X[:, known_indices]

def add_knowns(X):
    Y = ground_truth_with_knowns.repeat(torch.Size([X.size(dim=0), 1]))
    Y[:, known_indices] = X[:, replace_indices]
    return Y

def cost(Y):
    Y_hat = add_knowns(Y).cpu().detach().numpy()
    C = []
    for x in Y_hat:
        C.append(float(cost_(x)))
    return torch.tensor(C, requires_grad=True).to(DEVICE)

def g_x(Y):
    Y_hat = add_knowns(Y).cpu().detach().numpy()
    Gs = []
    for x in Y_hat:
        G = []
        for eqn in inequality_funs:
            # print(eqn, "=", eqn(x))
            G.append(float(eqn(x)))
        Gs.append(G)
    return torch.tensor(Gs, requires_grad=True).nan_to_num(0).to(DEVICE)

def ineq_grad(Y):
    Y_hat = add_knowns(Y).cpu().detach().numpy()
    Gs = []
    for x in Y_hat:
        Gs.append(np.array(del_ineq_fn(x)).flatten())
    return torch.tensor(Gs, requires_grad=True).nan_to_num(0).to(DEVICE)
        

KeyboardInterrupt: 

In [ ]:
print(cost(ground_truth))
print(g_x(ground_truth))
print(ineq_grad(ground_truth))


tensor([1353.4351], device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([[ -2.5000, -20.7632, -24.2368,  -2.5000, -28.8281, -16.1719,  -2.5000,
         -27.9531, -17.0469,  -2.5000, -22.1447, -22.8553,  -0.9416,  -4.5873,
         -40.4127,  -2.6359, -26.7217, -18.2783,  -3.0447, -22.2393, -22.7607,
          -4.7280, -20.3357, -24.6643,  -3.9000, -23.0398, -21.9602,  -5.3253,
         -21.2877, -23.7123,  -3.2379, -24.7973, -20.2027,  -2.4670, -22.2595,
         -22.7405,  -0.9925,  -4.2361, -40.7639,  -2.2650,  -4.6799, -40.3201,
          -3.1976,  -5.2024, -39.7976,  -3.9820,  -5.2909, -39.7091,  -1.5807,
          -4.7393, -40.2607,   0.0000,  -3.5000,   0.0000,  -3.5000,   0.0000,
          -3.5000,   0.0000,  -3.5000]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0', dtype=torch.float64, grad_fn=<ToCopyBackward0>)


In [ ]:
G = np.random.normal(loc=0, scale=1., size=(5, 3))
print(G)

[[-0.04979815  0.63972217  0.13694563]
 [ 0.63419678  1.08472149  1.44420107]
 [ 0.39851485 -0.23787081  0.06135996]
 [ 0.66401494 -1.29885013 -0.2292368 ]
 [-0.7709291   1.96366343  0.03149286]]


In [ ]:
class OptimizerNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        layers = [
            nn.Linear(input_dim, hidden_dim), 
            nn.BatchNorm1d(hidden_dim), 
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(hidden_dim, hidden_dim), 
            nn.BatchNorm1d(hidden_dim), 
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(hidden_dim, output_dim)
        ]
        
        for layer in layers:
            if type(layer) == nn.Linear:
                nn.init.kaiming_normal_(layer.weight)

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

n_train = 10000
n_test = 100
train_batch_size = 1000
train_X = torch.Tensor(np.random.uniform(-1, 1, size=(n_train, n_unknowns))).to(DEVICE)
test_X = torch.Tensor(np.random.uniform(-1, 1, size=(n_test, n_unknowns))).to(DEVICE)

train_dataset = TensorDataset(train_X)
test_dataset = TensorDataset(test_X)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1)


In [ ]:
for Xtrain in train_loader:
    Xtrain = Xtrain[0].to(DEVICE)
    print(g_x(Xtrain))
    

tensor([[ 8.0597e-01, -1.4765e+01, -3.0235e+01,  ..., -1.0425e+00,
          9.9937e+00, -1.3494e+01],
        [ 7.5994e-01,  1.0953e+01, -5.5953e+01,  ..., -6.7351e-01,
          1.0109e+01, -1.3609e+01],
        [ 8.0489e-02,  2.1376e+01, -6.6377e+01,  ...,  1.5835e-01,
          1.1236e+01, -1.4736e+01],
        ...,
        [ 7.0389e-01,  1.9305e+02, -2.3805e+02,  ...,  3.8483e-02,
          1.0126e+01, -1.3626e+01],
        [ 6.5772e-01,  1.6313e+01, -6.1313e+01,  ...,  7.8095e-02,
          1.0257e+01, -1.3757e+01],
        [ 4.6090e-01, -3.5295e+01, -9.7045e+00,  ...,  5.6120e-01,
          1.0715e+01, -1.4215e+01]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([[ 6.2648e-01,  6.2811e+01, -1.0781e+02,  ..., -5.3202e-01,
          9.5018e+00, -1.3002e+01],
        [ 6.4881e-01,  1.6779e+02, -2.1279e+02,  ..., -7.0115e-01,
          1.0473e+01, -1.3973e+01],
        [ 6.0071e-01,  1.0062e+00, -4.6006e+01,  ..., -1.0517e-01,
          1.1283e+01, -1.4783e+01],
        

In [ ]:
def loss(Y, args):
    obj_cost = cost_fn(Y)
    ineq_cost = torch.linalg.vector_norm(torch.clamp(g_x(Y)), min=0)

    return obj_cost + args['softWeight'] * ineq_cost


def grad_step(X, args):
    gamma = args['ineqGradLr'] 
    steps = args['ineqGradSteps'] 
    momentum = args['ineqGradMomentum'] 
    old_x_step = 0
    X_new = X
    for i in range(steps):
        x_step = del_ineq(X_new)
        step = gamma * x_step + momentum * old_x_step
        X_new = X_new - step
        old_x_step = step
    
    return X_new


def eval_dataset(net, loader, args):
    avg_loss = 0
    n = 0
    for X in loader:
        n += 1
        X = X[0]
        loss, Y_new = eval(net, X, args)
        avg_loss += loss
        # print('Test set {0} loss = {1} Y_new = {2}'.format(i, loss, Y_new))
    return avg_loss / n

def eval(net, X, args):
    # print(X)
    Y = net(X)
    Y_new = grad_step(Y, args)
    return loss(Y_new, args), Y_new

def train(net, train_loader, test_loader, args):
    solver_opt = optim.Adam(net.parameters(), lr=args['trainLr'])
    for epoch in range(args['epochs']):
        net.train()
        for i, Xtrain in enumerate(train_loader):
            solver_opt.zero_grad()
            Xtrain = Xtrain[0].to(DEVICE)
            Yhat_train = net(Xtrain)
            Ynew_train = grad_step(Yhat_train, args)
            train_loss = loss(Ynew_train, args)
            train_loss.sum().backward()
            solver_opt.step()
        net.eval()
        test_loss = eval_dataset(net, test_loader, args)
        print('Epoch = {0} loss = {1}'.format(epoch, test_loss))


In [ ]:
net = OptimizerNet(n_unknowns, 200, n_unknowns).to(DEVICE)
args = {}

args['softWeight'] = 10
args['trainLr'] = 0.001
args['ineqGradLr'] = 0.001
args['ineqGradSteps'] = 10
args['ineqGradMomentum'] = .5
args['epochs'] = 1000

broken_test = [3.5, 7, 10.4, -6.6+7, -4.1+7, 0, 4.9+7, 6.4+7, -1.9, -4, -4.9, -4, -1.9]
# X = grad_step(broken_test, 1e-4, 0.5, 10)
# print(X)
# print(cost_fn(X))
# print(torch.linalg.vector_norm(torch.clamp(g_x(np.array(broken_test).flatten()), min=0)))
# print(torch.linalg.vector_norm(torch.clamp(g_x(X.flatten()), min=0)))

# eval_dataset(net, test_loader, args)
train(net, train_loader, test_loader, args)

net.eval()
for test in test_loader:
    print(net(test[0]))
    print(cost_fn(net(test[0]).cpu().detach().numpy().flatten()))

NameError: name 'del_ineq' is not defined